In [1]:
import pandas as pd
from numpy import nan as NA
import numpy as np

In [2]:
file_name = str(input("enter the name of the file: "))

enter the name of the file: MHC-537-17 Classified Junction Count - All Sites


In [3]:
data_combined = pd.concat(pd.read_excel(file_name + '.xlsx',ignore_index=True, sheet_name=None, header=26))
data_raw = pd.read_excel(file_name + '.xlsx', sheet_name=None, ignore_index=True)
hourly_totals_row = data_combined[data_combined['TIME'] == 'Hourly Total']
movement_hourly_pcus = hourly_totals_row['PCU TOTAL']

In [4]:
hourly_pcus_arr = np.array(movement_hourly_pcus)

In [5]:
sheet_list = list(data_raw.keys())

In [6]:
movement_list = []
for sheet in sheet_list:
        if 'Movement' in sheet:
            movement_list.append(sheet)
number_of_movements = len(movement_list)

In [7]:
remove_variables = ['Hourly Total', 'Hourly Average', 'Session Total', 'Session Average', 'TIME']

for i in remove_variables:
    data_combined = data_combined.drop(data_combined[(data_combined['TIME'] == i)].index)
    
cleaned_PCU_totals = data_combined['PCU TOTAL'].dropna()

In [8]:
arr_totals = np.array(cleaned_PCU_totals)
number_of_time_intervals = len(arr_totals)/number_of_movements
organised_PCU_totals = arr_totals.reshape(number_of_movements, int(number_of_time_intervals))

In [9]:
survey_time_intervals = ["a","b","c","d","e","f", "g"]
number_of_time_intervals = int(input("enter the number of intervals: "))
survey_time_intervals = survey_time_intervals[:(number_of_time_intervals)]
split_locations = []
split_index = 0
for interval in range(number_of_time_intervals-1):
    split_index = (int(input("how many hours in interval " + str(interval)))*4) + split_index
    split_locations.append(split_index)

split_locations
survey_intervals = np.hsplit(organised_PCU_totals,split_locations)

enter the number of intervals: 3
how many hours in interval 02
how many hours in interval 12


In [10]:
rolling_hour_totals =[]
for i in survey_intervals:
    quarterly_sum = np.sum(i , axis=0)
    quarterly_sum_df = pd.DataFrame(quarterly_sum)
    rolling_hour_totals.append(quarterly_sum_df.rolling(4,).sum().dropna())

In [11]:
print( "The network hourly totals (in PCUs) are: \n" + str(rolling_hour_totals))

The network hourly totals (in PCUs) are: 
[         0
3  14438.0
4  14796.4
5  14949.5
6  15223.0
7  15647.3,          0
3  15443.5
4  15374.3
5  15166.7
6  14845.1
7  14950.1,          0
3  15574.9
4  15864.3
5  16068.9
6  16581.5
7  16560.2]


In [12]:
split_locations

[8, 16]